In [9]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.multioutput import ClassifierChain
from sklearn.svm import SVC
from gensim.models.doc2vec import Doc2Vec
import re
import jieba
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib 
from sklearn.linear_model import LogisticRegression
import random
from sklearn import tree

divorce_model=Doc2Vec.load("divorce/divorce.model") #divorce的Doc2Vec模型
loan_model=Doc2Vec.load("loan/loan.model")#loan的Doc2Vec模型
labor_model=Doc2Vec.load("labor/labor.model")#labor的Doc2Vec模型

data_input = {1: "divorce/data.txt", 2: "labor/data.txt", 3: "loan/data.txt"}
models={1:divorce_model,2:loan_model,3:labor_model}
data_type = {1: "divorce", 2: "labor", 3: "loan"}
label_size = 20
test_ratio = 0.3
punction = "！？。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

def line_processing(line):  # 提取每行数据的文本内容
    line = line.strip().split('\t')
    sentence = line[1]
    sentence = re.sub(r'[{}]'.format(punction),' ',sentence).split(' ')
    sent=[]
    for sub_sentence in sentence:
        if sub_sentence!='':
            sent.extend(list(jieba.cut(sub_sentence)))
    return line[0], sent, line[2]

def constructDataSet(data_path,model_tag): #构建X，Y的数据集
    data_file = open(data_path,'r',encoding='utf-8')
    lines = data_file.read().splitlines()
    X=[]
    Y=[]
    d2v = models[model_tag]
    for line in lines:
        _,x,y = line_processing(line)
        x=d2v.infer_vector(x)
        X.append(x)
        y = list(map(int,y.split()))
        Y.append(y)
    Y = np.array(Y)
    data_file.close()
    return X,Y

def splitDataSet(X,Y,modify=False,ratio=.2): #构建训练集和测试集
    if modify: #调整正负样本比例
        X_true=[]
        X_false=[]
        Y_true=[]
        Y_false=[]
        for i in range(len(Y)):
            if Y[i]==1:
                X_true.append(X[i])
                Y_true.append(Y[i])
            else:
                X_false.append(X[i])
                Y_false.append(Y[i])
        true_num = len(X_true)
        false_num = true_num*ration
        for i in range(0,len(X_false),int(len(X_false)/false_num)):
            X_true.append(X_false[i])
            Y_true.append(Y_false[i])
        X_train,X_test,Y_train,Y_test =  train_test_split(X_true,Y_true,test_size=test_ratio,shuffle=True)
    else:
        X_train,X_test,Y_train,Y_test =  train_test_split(X,Y,test_size=test_ratio,shuffle=True)
    return X_train,X_test,Y_train,Y_test

def beginTrain():
    for i in range(1,4):
        print(data_input[i].split("/")[0])
        X,Y = constructDataSet(data_input[i],i)
        X_train,X_test,Y_train,Y_test = splitDataSet(X,Y)
        base_classifier=LogisticRegression(solver='lbfgs')
        chains = [ClassifierChain(base_classifier,order='random',random_state=j) for j in range(label_size**2)]
        for chain in chains:
            chain.fit(X_train,Y_train)
        chain_predict = np.array([chain.predict(X_test) for chain in chains])
        predict_label = chain_predict.mean(axis=0)
        for o in range(len(predict_label)):
            for j in range(len(predict_label[o])):
                predict_label[o][j]= 1 if predict_label[o][j]>0 else 0
        Y_test = Y_test.transpose()
        predict_label = predict_label.transpose()
        for j in range(label_size):
            f1socre=f1_score(Y_test[j],predict_label[j],average='macro')
            print(j+1,f1socre)

In [79]:
beginTrain()#20个分类器，NB分类器

divorce
0.07626546143558602
labor
0.035617921432096654
loan
0.0737285926476192


In [6]:
beginTrain()#20个分类器，NB分类器,单个标签评判

divorce
1 0.4707119868231133
2 0.4125705272336758
3 0.09465622579812409
4 0.2998749135189886
5 0.04424055232558139
6 0.05308938707689555
7 0.09859901509279102
8 0.3209122480763624
9 0.16116064843053307
10 0.3166234596010981
11 0.010756505053346008
12 0.14547543979306884
13 0.01322952880996587
14 0.008575197889182058
15 0.005764505764505765
16 0.13114890369376964
17 0.06325855495853752
18 0.01707450564673475
19 0.28364978120720463
20 0.006327918398186626
labor
1 0.06194690265486725
2 0.04845338524767467
3 0.045355902777777776
4 0.03159053384700056
5 0.03222967770322297
6 0.03127064523232768
7 0.04216040564212065
8 0.024070992789794785
9 0.021811928817026795
10 0.1275057503666432
11 0.40182463782129074
12 0.12645385999209402
13 0.3298674446047271
14 0.1288359062216717
15 0.27802160080659033
16 0.00668850161494771
17 0.49997158283603294
18 0.13853126921318515
19 0.189754724298761
20 0.2621778337450883
loan
1 0.10817451228438857
2 0.07560868050729525
3 0.0735531846104106
4 0.07400226207788

In [81]:
beginTrain()#400个分类器，NB分类器

divorce
0.07200562236679031
labor
0.03570615518887404
loan
0.07031495819423016


In [2]:
beginTrain()#20个分类器，logisticsRegression分类器

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\RPJ\AppData\Local\Temp\jieba.cache


divorce


Loading model cost 0.802 seconds.
Prefix dict has been built succesfully.


0.06466324685235987
labor
0.10627038221481713
loan
0.21205181405585583


In [8]:
beginTrain()#20个分类器，logisticsRegression分类器，单标签评判

divorce
1 0.5532116628033876
2 0.5826709602816835
3 0.5691591287585157
4 0.5612353208671629
5 0.5987122370246376
6 0.4976117373381455
7 0.5130593151200737
8 0.5635524446229719
9 0.4981907721478984
10 0.5371917821250947
11 0.49718027145861216
12 0.5019167764930602
13 0.5240951928926556
14 0.497324414715719
15 0.49876131491186276
16 0.49840286054827176
17 0.4990477097419293
18 0.4985224022878932
19 0.4982593352091182
20 0.49833110814419224
labor
1 0.6591267905337903
2 0.5386840871517753
3 0.6804379410543794
4 0.5352824523685966
5 0.5688013428715875
6 0.623272721478948
7 0.5897410495371109
8 0.5225792781246124
9 0.6362508785711333
10 0.5815526417214035
11 0.4997441291863308
12 0.49957340310562537
13 0.49980101199613397
14 0.4998294485503127
15 0.49977257220832383
16 0.49968723343758886
17 1.0
18 0.4997156829296031
19 0.4998294485503127
20 0.4998578818714115
loan
1 0.7463625354224475
2 0.6764056726004124
3 0.7000095205072077
4 0.6522484341656081
5 0.6678894000275708
6 0.6095591718276812
7 

In [4]:
beginTrain()#400个分类器，logisticsRegression分类器

divorce
0.08408771975212033
labor
0.10651358783314406
loan
0.2230398707901286


In [10]:
beginTrain()#400个分类器，logisticsRegression分类器，单标签评判

divorce
1 0.5844701163354225
2 0.6057946420682251
3 0.5636235908309304
4 0.5784833041337025
5 0.6083954383483497
6 0.5015242830724086
7 0.5151864946753857
8 0.5792959061938732
9 0.49497778636275835
10 0.5315150840476456
11 0.4969639015060961
12 0.4960482827992527
13 0.5103008506576602
14 0.49768441155407017
15 0.49840286054827176
16 0.4981875417342364
17 0.4991669443518827
18 0.49902385600685684
19 0.4986418870621873
20 0.4984267734553776
labor
1 0.6453634504153427
2 0.5457634616342241
3 0.677925679960636
4 0.547594044758508
5 0.5888095974833037
6 0.6305749354005168
7 0.5809478447249344
8 0.5384357932966071
9 0.645251572113491
10 0.6033558477327126
11 0.49965878070973613
12 0.4992886005349723
13 0.4998294485503127
14 0.4998294485503127
15 0.4998578818714115
16 0.49968723343758886
17 0.49997158283603294
18 0.4997441291863308
19 0.4998578818714115
20 0.4996303247454928
loan
1 0.7375897977070851
2 0.6743951534914667
3 0.7087260753777211
4 0.6578843043719764
5 0.7014773291785213
6 0.625624